<a href="https://colab.research.google.com/github/Arulsha26/-Water-potability-prediction-using-svm/blob/main/Q_%26_A_chatbot_using_own_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 22.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,TrainingArguments, Trainer,BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset

In [ ]:
data = {
    "text": [
        "### Question: What is AI?\n### Answer: AI means Artificial Intelligence.",
        "### Question: What is ML?\n### Answer: ML means Machine Learning.",
        "### Question: What is LoRa?\n### Answer: LoRa is efficient fine tunning."
    ]
}

In [ ]:
dataset = Dataset.from_dict(data)

In [ ]:
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize(batch):
  tokens = tokenizer(
      batch["text"],
      padding="max_length",
      truncation = True,
      max_length = 128
  )
  tokens["labels"] = tokens["input_ids"].copy()
  return tokens

tokenized_dataset = dataset.map(tokenize, batched=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"  # Required for 8-bit loading
)

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.1,
    bias="none",
    task_type = TaskType.CAUSAL_LM
)

In [ ]:
model= get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    learning_rate = 2e-4,
    logging_steps = 1,
    save_strategy="epoch",
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset = tokenized_dataset,
    tokenizer=tokenizer
)
trainer.train()